In [3]:
#import libraries
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

C:\Users\klpna\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:
train_dir = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/train",
    shuffle=True,
    image_size=(256, 256),
    batch_size=32
)


Found 5216 files belonging to 2 classes.


In [5]:
test_dir = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/test",
    shuffle=True,
    image_size=(256, 256),
    batch_size=32
)

Found 624 files belonging to 2 classes.


In [9]:
train_dir = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/val",
    shuffle=True,
    image_size=(256, 256),
    batch_size=32
)

Found 47 files belonging to 2 classes.


In [7]:
class_names = train_dir.class_names
class_names

['NORMAL', 'PNEUMONIA']

In [11]:
len(train_dir)

163

In [12]:
163*32

5216

In [17]:
for image_batch,label_batch in trai_dir.take(1):
    for i in range(12):
        ax=plt.subplot(3,4,i+1)
        plt.imshow(image_batch[0].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

NameError: name 'trai_dir' is not defined

In [36]:
len(dataset)

163

In [13]:
train_dir = train_dir.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dir = val_dir.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dir = test_dir.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [61]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(256, 256),
    layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])


Prepare the model

In [62]:
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_dir,
    epochs=5,
    validation_data=val_dir,
    batch_size=32,
    verbose=1
)


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.7536 - loss: 0.5923 - val_accuracy: 0.5319 - val_loss: 0.6412
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 140s 856ms/step - accuracy: 0.8679 - loss: 0.3408 - val_accuracy: 0.7447 - val_loss: 0.5638
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 155s 952ms/step - accuracy: 0.8901 - loss: 0.3091 - val_accuracy: 0.8936 - val_loss: 0.4705
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 642s 4s/step - accuracy: 0.9022 - loss: 0.2396 - val_accuracy: 0.6383 - val_loss: 0.7783
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 705s 4s/step - accuracy: 0.9204 - loss: 0.1994 - val_accuracy: 0.8723 - val_loss: 0.3478


In [55]:
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)            │ (32, 255, 255, 2)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_3 (Sequential)            │ (32, 255, 255, 2)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (32, 253, 253, 32)          │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ (32, 126, 126, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (32, 124, 124, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_27 (MaxPooling2D)      │ (32, 62, 62, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_13 (Flatten)                 │ (32, 246016)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (32, 64)                    │      15,745,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (32, 1)                     │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,764,257 (60.14 MB)

 Trainable params: 15,764,257 (60.14 MB)

 Non-trainable params: 0 (0.00 B)

In [63]:
##plot the history curve

In [64]:
##test some samples

In [ ]:
model.evaluate(test_dir)

20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 642ms/step - accuracy: 0.8173 - loss: 0.4619
